## Spectrogram like audiovectorizer ##

In [ ]:
import torch
import torchaudio


torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
audio_file = '/content/drive/My Drive/000004_000000.wav'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import IPython
IPython.display.Audio(audio_file)

In [ ]:
wav, sr = torchaudio.load(audio_file)

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [ ]:
mel_spec = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_fft=256)(wav)

/usr/local/lib/python3.12/dist-packages/torchaudio/functional/functional.py:585: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (129) may be set too low.
  warnings.warn(


In [ ]:
mel_spec.size()

torch.Size([1, 128, 539])

Sequence to vector example wit softmax sum

In [ ]:
class SpecToVec(torch.nn.Modue):
    def __init__(
            self,
            in_channels: int = 128,
            out_channels: int = 256,
            kernel: int = 10,
            out_features: int = 512
        ):
        super().__init__()

        self.conv = torch.nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
                                    kernel_size=10, stride=4)
        self.key_proj = torch.nn.Linear(in_features=out_channels, out_features=out_features)
        self.q_proj = torch.nn.Linear(in_features=out_features, out_features=1)

    def forward(self, spec):
        spec = spec.view(0,2,1)
        x = self.conv(spec)
        k = self.key_proj(x)
        q = self.q_proj(x)
        out = torch.sum(torch.nn.Softmax(q) * k, dim=-1)

        return out

